In [2]:
import pandas as pd
import numpy as np
import os

# Merge Kaggle and Git Data

In [5]:
### Merging merged_kaggle.csv and games.csv

# Read the first and second datasets
df_nfl = pd.read_csv('./data-raw-kaggle-1/merged_kaggle.csv')
df_games = pd.read_csv('./data-raw-git-1/games.csv')

# Convert the 'schedule_week' column to integers where possible
df_nfl.loc[df_nfl['schedule_week'].str.isnumeric(), 'schedule_week'] = df_nfl.loc[df_nfl['schedule_week'].str.isnumeric(), 'schedule_week'].astype(int)

# Handle playoff weeks for 2021 season
playoff_week_mapping_2021 = {
    'Wildcard': 19,
    'Division': 20,
    'Conference': 21,
    'Superbowl': 22
}
mask_2021 = (df_nfl['schedule_season'] == 2021) & (df_nfl['schedule_week'].isin(playoff_week_mapping_2021.keys()))
df_nfl.loc[mask_2021, 'schedule_week'] = df_nfl.loc[mask_2021, 'schedule_week'].replace(playoff_week_mapping_2021)

# Handle playoff weeks for 2022 season
playoff_week_mapping_2022 = {
    'Wildcard': 19,
    'Division': 20,
    'Conference': 21,
    'Superbowl': 22
}
mask_2022 = (df_nfl['schedule_season'] == 2022) & (df_nfl['schedule_week'].isin(playoff_week_mapping_2022.keys()))
df_nfl.loc[mask_2022, 'schedule_week'] = df_nfl.loc[mask_2022, 'schedule_week'].replace(playoff_week_mapping_2022)

# Handle playoff weeks
playoff_week_mapping = {
    'Wildcard': 18,
    'Division': 19,
    'Conference': 20,
    'Superbowl': 21
}
df_nfl['schedule_week'].replace(playoff_week_mapping, inplace=True)

# Create a mapping dictionary only for the teams that got renamed or relocated
team_name_mapping = {
    'STL': 'LAR',  # St. Louis Rams to Los Angeles Rams
    'OAK': 'LVR',  # Oakland Raiders to Las Vegas Raiders
    'LV': 'LVR',   # LV to LVR just cause
    'SD': 'LAC',   # San Diego Chargers to Los Angeles Chargers
    'LA': 'LAR'    # LA could refer to Los Angeles Rams
}

df_games['home_team'] = df_games['home_team'].replace(team_name_mapping)
df_games['away_team'] = df_games['away_team'].replace(team_name_mapping)

# Rename columns in df_games
rename_columns = {
    'weekday': 'weekday_GIT',
    'gametime': 'gametime_GIT',
    'away_qb_id': 'away_qb_id_GIT',
    'home_qb_id': 'home_qb_id_GIT',
    'away_rest': 'away_rest_GIT',
    'home_rest': 'home_rest_GIT',
    'away_moneyline': 'away_moneyline_GIT',
    'home_moneyline': 'home_moneyline_GIT',
    'spread_line': 'spread_line_GIT',
    'away_spread_odds': 'away_spread_odds_GIT',
    'home_spread_odds': 'home_spread_odds_GIT',
    'total_line': 'total_line_GIT',
    'under_odds': 'under_odds_GIT',
    'over_odds': 'over_odds_GIT',
    'away_qb_name': 'away_qb_name_GIT',
    'home_qb_name': 'home_qb_name_GIT',
    'away_coach': 'away_coach_GIT',
    'home_coach': 'home_coach_GIT',
    'referee': 'referee_GIT'
}
df_games_renamed = df_games.rename(columns=rename_columns)

# Columns to merge
columns_to_merge = list(rename_columns.values()) + ['season', 'week', 'home_team', 'away_team']

# Perform the merge
df_merged = pd.merge(df_nfl, df_games_renamed[columns_to_merge], 
                     left_on=['schedule_season', 'schedule_week', 'home_team_id', 'away_team_id'],
                     right_on=['season', 'week', 'home_team', 'away_team'],
                     how='left')

# Drop unnecessary columns
# df_merged.drop(['home_team', 'away_team'], axis=1, inplace=True)

# Save the merged DataFrame
df_merged.to_csv('./data-final/nfl_games_final.csv', index=False)

# Create unique_game_id for unmatched games check
df_games['unique_game_id'] = df_games['season'].astype(str) + '_' + df_games['week'].astype(str) + '_' + df_games['home_team'] + '_' + df_games['away_team']
df_merged['unique_game_id'] = df_merged['schedule_season'].astype(str) + '_' + df_merged['schedule_week'].astype(str) + '_' + df_merged['home_team_id'] + '_' + df_merged['away_team_id']

# Find unmatched games
unmatched_games = df_games[~df_games['unique_game_id'].isin(df_merged['unique_game_id'])]

# Print the number of unmatched rows
print(f"Number of rows in games.csv that did not find a corresponding row in merged DataFrame: {len(unmatched_games)}")


Number of rows in games.csv that did not find a corresponding row in merged DataFrame: 6


In [4]:
### Print the missing rows

# Selected columns to display for easier analysis
selected_columns = ['season', 'game_type', 'week', 'gameday', 'weekday', 'home_team', 'away_team', 'unique_game_id']

# Filter the unmatched games with the selected columns
unmatched_games_final_table = unmatched_games[selected_columns]

# Display the table
unmatched_games_final_table


season game_type  week     gameday weekday home_team away_team  \
258     1999        SB    21  2000-01-30  Sunday       TEN       LAR   
517     2000        SB    21  2001-01-28  Sunday       NYG       BAL   
776     2001        SB    21  2002-02-03  Sunday        NE       LAR   
1043    2002        SB    21  2003-01-26  Sunday        TB       LVR   
1609    2005       REG     2  2005-09-19  Monday        NO       NYG   
2111    2006        SB    21  2007-02-04  Sunday       CHI       IND   

       unique_game_id  
258   1999_21_TEN_LAR  
517   2000_21_NYG_BAL  
776    2001_21_NE_LAR  
1043   2002_21_TB_LVR  
1609    2005_2_NO_NYG  
2111  2006_21_CHI_IND